In [1]:
# Magic
%matplotlib inline

# Libraries in use
import os
import sys
import math
import pylab as pl
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn import linear_model, neural_network, metrics
from matplotlib import collections  as mc
import string

from myutil import *
import pickle

In [2]:
with open('assetslist_lasso_trim_10_300.pckl') as f:
    assetslist = pickle.load(f)

In [3]:
# Read the data
df = pd.read_csv("../../data/test.csv.gz")

In [4]:
"Number of securities in use is {}".format(len(set(df['ID'])))

'Number of securities in use is 1136'

In [5]:
"Number of days in train dataset {}".format(len(set(df['TIMESTAMP']))) 

'Number of days in train dataset 458'

In [6]:
timestamps = sorted(list(set(df['TIMESTAMP'])))

In [7]:
df_result = df.loc[:, ['TIMESTAMP', 'ID']].copy(deep=True)

In [8]:
y_pred = []
data = AssetsData()
print y_pred

[]


In [9]:
for time in timestamps:
    df_current = df[(df['TIMESTAMP'] == time)]
    ids = list(df_current['ID'])
    for id_current in ids:
        df_current_id =  df_current[(df_current['ID'] == id_current)]
        col = data.get_factor_names(df)
        current_id_factors = df_current_id[col]
        if id_current not in assetslist:
            y_pred.append(0)
            #print 'Not found'
            continue
        else:    
            asset = assetslist[id_current]
        if type(asset.factors) != pd.core.indexes.base.Index:
            y_pred.append(0)
            #print 'Not found factors'
            continue
            
        if asset.score < 0.03:
            y_pred.append(0)
            continue
        #print current_id_factors[asset.factors]
        current_id_factors_wout_nan = current_id_factors[asset.factors].fillna(0)
        x = np.array(current_id_factors_wout_nan)
        X = (x - asset.mean)/asset.var
        
        next_pred = asset.model.predict(X)
        #if next_pred > 1e+2 or next_pred < -1e+2:
            #y_pred.append(0)
            #continue
        
        #asset.mean = (asset.mean * asset.day_count + x)/(asset.day_count + 1)
        #asset.day_count += 1
        
        y_pred.append(next_pred)
        #print np.array(current_id_factors[asset.factors])
        #print asset.mean
        #print (x - asset.mean)/asset.var
        #print asset.model.predict(x)
        #print current_id_factors
        #print time, id_current
        

In [10]:
print max(y_pred)
print min(y_pred)
y_pred_hat = [float(i) for i in y_pred]
len(y_pred_hat)

[ 77405.54852119]
[-8765.82456505]


463691

In [31]:
import copy
y_pred_hat_trim = copy.deepcopy(y_pred_hat)
for ind, j in  enumerate(y_pred_hat_trim):
    if j > 1.5 or j < -1.5:
        y_pred_hat_trim[ind] = 0

In [ ]:
import copy
y_pred_hat_trim = copy.deepcopy(y_pred_hat)
for ind, j in  enumerate(y_pred_hat_trim):
    if j > 1:
        y_pred_hat_trim[ind] = 1
    if j < -1:
        y_pred_hat_trim[ind] = -1

In [23]:
print max(y_pred)
print min(y_pred)
print max(y_pred_hat)
print min(y_pred_hat)
print max(y_pred_hat_trim)
print min(y_pred_hat_trim)

[ 77405.54852119]
[-8765.82456505]
77405.5485212
-8765.82456505
2.97001490997
-2.99972340223


In [32]:
df_result.loc[:, 'y_pred'] = y_pred_hat_trim

In [33]:
df_result.head()

,TIMESTAMP,ID,y_pred
0,T1122,S0001,0.000000
1,T1122,S0002,0.000000
2,T1122,S0003,0.000586
3,T1122,S0004,0.000000
4,T1122,S0005,0.001723


In [34]:
df_result.shape == (463691, 3)

True

In [35]:
(df_result[['TIMESTAMP','ID']] == df[['TIMESTAMP','ID']]).all()

TIMESTAMP    True
ID           True
dtype: bool

In [36]:
df_result.to_csv('out1.csv', index=False)

In [ ]:
nmgbhjgfghf

In [ ]:
wgtcol='y_pred'
dlywgts = df_result.groupby('TIMESTAMP')[wgtcol].apply(
                lambda x: x.abs().sum()).reset_index().rename(columns={wgtcol:'dailywgt'})

In [ ]:
df_result = df_result.merge(dlywgts)
df_result['wgt'] = df_result[wgtcol] / df_result['dailywgt']
pt = df_result.pivot_table('wgt',index='TIMESTAMP',columns='ID')
turnover = (pt - pt.shift(1)).abs().sum(axis=1).mean()
print turnover

In [ ]:
for i in timestamps:
    if sum(abs(df_result[(df_result['TIMESTAMP'] == i)].loc[:, 'wgt']) < 1e-5) > len(df_result[(df_result['TIMESTAMP'] == i)].loc[:, 'wgt'])/2:
        print 'fail'


In [ ]:
for i, j in enumerate(y_pred_hat):
    if j > 100:
        print i

In [29]:
df_result.loc[:, 'y_pred'] = y_pred_hat

In [30]:
df_result.to_csv('out1.csv', index=False)